In [1]:
##############################
# import modules
##############################

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

from gensim import models

In [4]:
##############################
# define functions
##############################

def length(sequence):
    """
    function that computes the real, unpadded lenghts for every sequence in batch
    """
    used = tf.sign(tf.reduce_max(tf.abs(sequence), reduction_indices=2))
    length = tf.reduce_sum(used, reduction_indices=1)
    length = tf.cast(length, tf.int32)
    return length

def mlp(_X, _weights, _biases):
    """
    function that defines a multilayer perceptron in the graph
    input shape: parse_steps (=?) x filtered_words (=3) x lstm_output_length (=400)
    output shape: parse_steps (=?) x num_classes
    """
    # ReLU hidden layer (output shape: parse_steps x n_hidden)
    layer_1 = tf.nn.relu(tf.add(tf.einsum('ijk,kl->il', _X, _weights['h']), _biases['b'])) 
    # return output layer (output shape: parse_steps x n_classes)
    return tf.add(tf.einsum('ik,kl->il', layer_1, _weights['out']), _biases['out'])

def embedding_lookup(sentences, max_seq_length, vec_length):
    """
    function that looks up embeddings.
    input: list of sentences, length of sentences, length of word vectors
    output: 3D array of word vectors per sentence 
                (dims #sentences x sentence_length x embedding_size)
    """
    sentence_embeddings = np.empty((0,max_seq_length,vec_length))
    for sentence in sentences:
        word_embeddings = np.empty((0,vec_length))
        for word in sentence:
            word_embeddings = np.vstack([word_embeddings, model[word]])
        if len(sentence) < max_seq_length:
            zero_padding_length = max_seq_length - len(sentence)
            word_embeddings = np.vstack([word_embeddings, np.zeros((zero_padding_length, vec_length))])
        sentence_embeddings = np.append(sentence_embeddings, np.array([word_embeddings]), axis=0)
    return sentence_embeddings


In [5]:
"""
Ik (Lucas) staar me er een beetje blind op nu, dus Niels, zou jij 
kunnen nadenken over onderstaande punten en sowieso even naar de 
code kunnen kijken?

TODO:

- Ik ga er nu vanuit dat ik een lijst van zinnen en een lijst van parses
terugkrijg. Ik weet niet goed hoe ik die moet opdelen in batches voor
training. De dummy data die ik nu gebruik moet een batch voorstellen.

- Tweede, gerelateerde issue: het model gaat nu er basically vanuit dat
het een batch ontvangt. Ik denk dus dat het het beste is om een groot
deel van de onderstaande code waarin ik de graph define (dus voordat
de session begint) te wrappen in een functie (de "train_op") met 
tf.placeholders die via een feed_dict worden gevuld met de huidige batch.

- Als ik vectoren probeer op te halen uit het model, maar dan voor 
woorden die er niet inzitten, dan krijg ik een error. Dit moeten we 
op een of andere manier zien te fixen.

- Ik zit even vast en weet niet goed hoe ik de uiteindelijke session
(= voornamelijk de training loop) die de graph runt, moet opbouwen. Er
staan voorbeelden van online, maar ik kan ze nog niet concreet toepassen
omdat ik nog geen concrete input data heb, en omdat ik nog niet weet hoe
ik batches maak/afhandel. DIT IS EEN BELANGRIJK PUNT NOG!!
"""


##############################
# build graph
##############################

with tf.Graph().as_default():
    
    # hyperparameters (from Cross & Huang, 2016)
    batch_size = 10
    n_input = 400
    n_hidden = 200
    n_classes = 50 # TODO: how many classes for the actual data?
    lstm_units = 200
    num_epochs = 10
    dropout = 0.5
    L2_penalty = 0.
    rho = 0.99
    epsilon = 1e-07

    # Store layers weight & bias
    weights = {
        'h': tf.Variable(tf.random_normal([n_input, n_hidden], dtype=tf.float64)),
        'out': tf.Variable(tf.random_normal([n_hidden, n_classes], dtype=tf.float64))
    }
    biases = {
        'b': tf.Variable(tf.random_normal([n_hidden], dtype=tf.float64)),
        'out': tf.Variable(tf.random_normal([n_classes], dtype=tf.float64))
    }

    # load word2vec model
    model_name = "dep_parser_word2vec_total"
    if not 'model' in locals():
        model = models.word2vec.Word2Vec.load(model_name)
        print("Model loaded from disk")
    else:
        print("Model was already loaded")

    
    # dummy data:
    # toy sentences (= list of lists of words)
    sentences = [["the", "by", "an", "on", "the", "in", "an"], ["the", "cat", "sat", "on", "the", "ground"]]
    
    # toy parses (= array with an action and word indices)
    parses = [np.array([[1,0,1,2], 
                        [0,1,2,3],
                        [0,1,2,3],
                        [1,0,1,2]]),
              np.array([[1,0,1,2], 
                        [0,1,2,3], 
                        [3,2,3,4]])]

    # variables from sentences
    vec_length = model['a'].size
    seq_lengths = [len(sentence) for sentence in sentences]
    max_seq_length = max(seq_lengths)
    

    # look up embeddings of words in sentence
    embeddings = embedding_lookup(sentences, max_seq_length, vec_length)
    print("Sentence embedding shape (np-array): ", embeddings.shape)

    # define LSTM cell + dropout wrapper (like Cross & Huang)
    cell = tf.nn.rnn_cell.BasicLSTMCell(num_units=lstm_units, state_is_tuple=True)
    cell = tf.nn.rnn_cell.DropoutWrapper(cell=cell, output_keep_prob=0.5)

    # define bidirectional architecture
    outputs, _ = tf.nn.bidirectional_dynamic_rnn(
        cell_fw=cell,
        cell_bw=cell,
        dtype=tf.float64,
        sequence_length=seq_lengths,
        inputs=embeddings
    )

    # fw/bw output (num_sequences x max_seq_length x lstm_units)
    output_fw, output_bw = outputs

    # concatenate forward & backward outputs per word
    output_lstm = tf.concat(2, outputs)
    print("BiLSTM output shape: ", output_lstm.get_shape())


    # MLP layer filtering LSTM outputs based on parse steps
    batch_cost = 0
    for i in range(0, len(sentences)):
        sentence, parse = output_lstm[i], parses[i]
        # input: parse_steps (=?) x filtered_words (=3) x lstm_output_length (=400)
        # output: parse_steps (=?) x num_classes
        output_mlp = mlp(tf.gather(sentence, parse[:,1:]), weights, biases)
        print("MLP output shape S{}: ".format(i), output_mlp.get_shape())
        cost = tf.reduce_sum(tf.nn.sparse_softmax_cross_entropy_with_logits(output_mlp, parse[:,0]))
        batch_cost += cost
    batch_cost /= len(sentences)

    # define optimizer
    optimizer = tf.train.AdadeltaOptimizer(rho=rho, epsilon=epsilon).minimize(batch_cost)

    
    ##############################
    # start session in graph
    ##############################

    init = tf.initialize_all_variables()

    with tf.Session() as sess:
        sess.run(init)
        
        # TODO: HOE WERKT TRAINING?


Model was already loaded
Sentence embedding shape (np-array):  (2, 7, 189)
BiLSTM output shape:  (2, 7, 400)
MLP output shape S0:  (4, 50)
MLP output shape S1:  (3, 50)
